In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import datetime
import gmaps
import os

# # Incorporated citipy to determine city based on latitude and longitude
# from citipy import citipy

# # Import API key
# from api_keys import g_key
# gmaps.configure(api_key=g_key)

In [2]:
# Make a reference to the starter csv file path and create df
prisoner_df = pd.read_csv('../Resources/Texas_Prison_Data.csv') 
prisoner_df.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/1/2020,11/15/2025,9/26/2018,68770,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,9/13/2017,8,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS
1,524524,758486,"HARPER,DAVID JOHN JR",Estelle,M,W,91,1/1/9999,1/1/9999,8/3/2031,95-765-C,McLennan,36010001,INDECENCY WITH A CHILD,8/13/1996,9/15/1995,Life,None,8/3/2031,NOT IN REVIEW PROCESS
2,703267,841623,"RODRIGUEZ,THEODORE",Beto,M,W,82,8/3/2043,8/3/2043,2/1/2021,97-396-C,McLennan,11990003,AGG SEX ASLT,8/5/1998,6/1/1996,45,None,2/1/2021,IN PAROLE REVIEW PROCESS
3,708520,256174,"ALONZO,BENITO",Hospital Galveston,M,H,85,1/1/9999,1/1/9999,11/25/1985,49995,Travis,49030000,INTRODUCE DEADLY WPN PENAL INSTI,2/27/1976,6/1/1974,Life,Denied on 12/20/2019,20-Dec,IN PAROLE REVIEW PROCESS
4,766785,365547,"BROWN,NARRIES EARL",Jester IV,M,W,84,1/1/9999,1/1/9999,8/24/2003,F83-89728-HI,Dallas,11220000,AGG SEX ABUSE CHILD/U14,8/24/1983,3/15/1983,Life,Denied on 12/05/2016,Unavailable at this time.,NOT IN REVIEW PROCESS


In [3]:
# prisoner_df.isna().sum()

In [4]:
# prisoner_df.dtypes

In [5]:
# This df is from the Texas Department of Criminal Justice (TDCJ)
# It has ALL the criminal code assignments - those that are currently being
# used AND those no longer used.  Some individuals have codes assigned that are no
# longer active.  This df has a column (Offense_Category)that can be used as a key 
# to link many different specific, detailed codes (Offense Codes) together.

offense_df = pd.read_csv('../Resources/offenseCodes_ut8.csv', encoding="utf-8")
# offense_df.head()

In [6]:
# offense_df.dtypes

In [7]:
# Create project specific df for merging. 
# This step creates a smaller dataframe that will be used to merge 
# with the prisoner df on the key offense cateogry

offproject_df = offense_df.loc[:,["Offense Code", "Offense_Category"]]
# offproject_df.head()

In [8]:
# This step actually creates the "bins" - the last column "Offense_Category" can be sorted on
# and capture the respective individuals by offense type.  The "ffill" will place a Nan in 
# the "Offense_Category" column if there is no value in that location.  
# The "merge_ordered" maintains the integrity of the elements in the order of the main df,
# prisoner_df, while matching the Offense_Cateogry" with the respective "Offense Code".
# The prisoner df has approximately 208K rows with many duplicated Offense Code.  
# The offproject df has 2,738 rows of 62 unique Offense Cateogries and related Offense Codes
# which results in the 2,738 rows on information in the df.  The Offense Code column is mapped
# to the respective value in the prisoner df and Offense_Category is assigned to the respective row.

binlookup_df = pd.merge_ordered(prisoner_df, offproject_df, fill_method='ffill', left_by='Offense Code')
binlookup_df.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status,Offense_Category
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/1/2020,11/15/2025,9/26/2018,...,Grayson,35990003,POSS CS 4-200G WID METH,12/14/2017,9/13/2017,8,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,5/2/2021,6/18/2033,3/8/2016,...,Dallas,35990003,UNLAW POSS WIT C/S-METHAM,4/6/2006,9/3/2004,25,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,1/1/9999,1/1/9999,8/22/2020,...,Williamson,35990003,DEL C/S,9/19/1995,9/28/1994,Life,Denied on 07/16/2020,21-Jul,NOT IN REVIEW PROCESS,2200
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,4/21/2028,3/11/2043,10/20/2020,...,Grayson,35990003,POSS CONT SUB PG1 4-200 WITD,3/12/2018,3/31/2017,25,Denied on 08/13/2020,21-Aug,NOT IN REVIEW PROCESS,2200
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,8/16/2025,6/27/2040,2/14/2018,...,Johnson,35990003,DEL CONT SUB 4G-200G,11/10/2015,5/5/2015,25,Denied on 02/06/2020,21-Feb,NOT IN REVIEW PROCESS,2200


In [9]:
binlookup_df.isna().sum()

SID Number                     0
TDCJ Number                    0
Name                           0
Current Facility               0
Gender                         0
Race                           0
Age                            0
Projected Release              0
Maximum Sentence Date         28
Parole Eligibility Date     9017
Case Number                   99
County                         0
Offense Code                   0
TDCJ Offense                   0
Sentence Date               1233
Offense Date                   0
Sentence (Years)               1
Last Parole Decision        7815
Next Parole Review Date    22397
Parole Review Status        7815
Offense_Category               0
dtype: int64

In [10]:
#Created this to keep from corrupting the good file
#did not need to do this - did not want issues a 2:37am
backup_df = binlookup_df.copy()

In [11]:
#This identifies the unique Offense Catrgory in the prisoner (newly creaked binlookup)
# df.  There are a total of 62 offense catories overall and 45 were matched to the prisoners
# in the prisoner df when creating the merged_ordered df.

binlookup_df["Offense_Category"].unique()

array([2200,  400, 3200,  200,  500, 4200,  600, 1500, 3700, 1900,  900,
       1600, 1800, 3500, 2300,  300, 4400, 4600, 4500,  700, 3400, 3100,
       4900, 3600, 1200, 2400, 2900, 2700,  100, 4100, 6200, 2100, 2600,
       4300, 3300, 3900, 1100, 2500, 6100, 5300, 1400, 5200, 4700, 5500,
       5400], dtype=int64)

In [12]:
# Create bins in which to place values based upon Offense Categories
# The step serves the purpose of assigning a label to the respective offense category
# must have 1 extra bin then actual catorgies - hence the 0 at the beginning

bins = [0, 100, 200, 300, 400, 500, 600, 700, 900, 1100, 1200, 1400, 
        1500, 1600, 1800, 1900, 2100, 2200, 2300, 2400, 2500, 2600, 
        2700, 2900, 3100, 3200, 3300, 3400, 3500, 3600, 3700, 3900, 
        4100, 4200, 4300, 4400, 4500, 4600, 4700, 4900, 5200, 5300, 
        5400, 5500, 6100, 6200]

# Create labels for these bins
group_labels = ["Human_Trafficking", "Murder", "Kidnapping", "Sexual_Assault", "Robbery", "Assault",
                "Coercion_Politics", "Harrassment", "Terrorism", "Arson", "Unlawful_Use", "Burglary", "Theft",
                "Unlawful_Rep", "Fraud_Forgery", "Vandalism", "Drugs", "Sexual_Related_Materials", "Child_Endanger", "Bigomy",
                "Elderly_at_Risk_Adult_Endanger", "Violate_Order", "Prostitution", "Evading_Arrest", "Criminal_Escape",
                "Perjury_Contemp", "Jump_Bail_No_Show", "Interference_Records", "Bribery", "Weapon_Related", 
                "Riot_Obstruction", "Vehicle_Incident", "Alcohol_Incident", "Animal_Related", "Agriculture_Related",
                "Health_Code", "Hazardous_Material", "Discrimination_Civil_Rights", "Record_Tampering", "Tax_Evasion",
                "Pollution", "Money_Laundering", "Negligence", "Business_License", "Organized_Crime"]

In [13]:
# Slice the data and place it into bins
# pd.cut(binlookup_df["Offense_Category"], bins, labels=group_labels).head()

In [14]:
# Place the data series (offense category label) into a new (last) column inside of the DataFrame

binlookup_df["Bin_Cat"] = pd.cut(binlookup_df["Offense_Category"], bins, labels=group_labels)
binlookup_df.head()

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,...,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status,Offense_Category,Bin_Cat
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/1/2020,11/15/2025,9/26/2018,...,35990003,POSS CS 4-200G WID METH,12/14/2017,9/13/2017,8,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,5/2/2021,6/18/2033,3/8/2016,...,35990003,UNLAW POSS WIT C/S-METHAM,4/6/2006,9/3/2004,25,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,1/1/9999,1/1/9999,8/22/2020,...,35990003,DEL C/S,9/19/1995,9/28/1994,Life,Denied on 07/16/2020,21-Jul,NOT IN REVIEW PROCESS,2200,Drugs
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,4/21/2028,3/11/2043,10/20/2020,...,35990003,POSS CONT SUB PG1 4-200 WITD,3/12/2018,3/31/2017,25,Denied on 08/13/2020,21-Aug,NOT IN REVIEW PROCESS,2200,Drugs
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,8/16/2025,6/27/2040,2/14/2018,...,35990003,DEL CONT SUB 4G-200G,11/10/2015,5/5/2015,25,Denied on 02/06/2020,21-Feb,NOT IN REVIEW PROCESS,2200,Drugs


In [15]:
rcbinlookup_df = binlookup_df.rename(columns={"SID Number": "SID_Number", 
                            "TDCJ Number": "TDCJ_Number",
                            "Current Facility" : "Current_Facility",
                            "Projected Release" : "Projected_Release",
                            "Maximum Sentence Date" : "Max_Sentence_Date",
                            "Parole Eligibility Date" : "Parole_Eligible_Date",
                            "Case Number" : "Case_Number",
                            "Offense Code" :"Offense_Code",
                            "TDCJ Offense" :"TDCJ_Offence",
                            "Sentence Date" : "Sentence_Date",
                            "Offense Date": "Offense_Date",
                            "Sentence (Years)" : "Sentence_Years",
                            "Last Parole Decision" : "Last_Parole_Decision",
                            "Next Parole Review Date" : "Next_Parole_Review",
                            "Parole Review Status" : "Parole_Review_Status"})

In [16]:
rcbinlookup_df["Sentence_Years"].unique()

array(['8', '25', 'Life', '12', '17', '15', '10', '18', '65', '22', '30',
       '35', '40', '60', '5', '9', '50', '16', '20', '3', '7', '23', '75',
       '70', '99', '14', '6', '45', '54', '13', '26', '28', '19', '24',
       '85', '33', '27', '49', '11', '29', '32', '55', '80', '9.5', '21',
       '13.5', '68', '90', '48', '78', '8.5', '37', '95', '4', '42', '43',
       '56', '38', '36', '12.5', '47', '2', '15.5', '77', '44', '0.7',
       '10.5', 'LWOP', '61', '2.5', '3.3', '12.6', '6.5', '3.5', '10.1',
       '1.5', '52', '7.5', '41', '52.5', '57', '31', '72', '53', '22.5',
       '17.5', '32.5', '62', '17.6', '130', '39', 'Capital Life', '198',
       '51', '119', '3000', '1000', '159', '98', '102', '114', '50.5',
       '58', '106', '46', '297', '67', '66', '104', '88', '69', '161',
       '76', '87', '146', '91', '37.5', '17.4', '82', '73', '63', '89',
       '27.5', '92', '34', '39.5', '18.5', '81', '26.5', '47.5', '59',
       '74', '16.5', '25.5', '18.3', '6.2', '14.5', '20

In [17]:
rcbinlookup_df.dtypes

SID_Number                 int64
TDCJ_Number                int64
Name                      object
Current_Facility          object
Gender                    object
Race                      object
Age                        int64
Projected_Release         object
Max_Sentence_Date         object
Parole_Eligible_Date      object
Case_Number               object
County                    object
Offense_Code               int64
TDCJ_Offence              object
Sentence_Date             object
Offense_Date              object
Sentence_Years            object
Last_Parole_Decision      object
Next_Parole_Review        object
Parole_Review_Status      object
Offense_Category           int64
Bin_Cat                 category
dtype: object

In [18]:
# rcbinlookup_df["Offense_Code"].unique()

In [19]:
rcbinlookup_df=rcbinlookup_df.replace(to_replace="Life",value="50.9")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="LWOP", value="100.9")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="Capital Life", value = "101.9") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="Death", value = "150.9")

#Not able to do NaN with this method
# Index 71997 SID_Number 4340086   TDCJ_Number 2319589 is the NaN
# rcbinlookup_df=rcbinlookup_df.replace(to_replace="NaN", value = "3999.9")

rcbinlookup_df.head()

,SID_Number,TDCJ_Number,Name,Current_Facility,Gender,Race,Age,Projected_Release,Max_Sentence_Date,Parole_Eligible_Date,...,Offense_Code,TDCJ_Offence,Sentence_Date,Offense_Date,Sentence_Years,Last_Parole_Decision,Next_Parole_Review,Parole_Review_Status,Offense_Category,Bin_Cat
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/1/2020,11/15/2025,9/26/2018,...,35990003,POSS CS 4-200G WID METH,12/14/2017,9/13/2017,8,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,5/2/2021,6/18/2033,3/8/2016,...,35990003,UNLAW POSS WIT C/S-METHAM,4/6/2006,9/3/2004,25,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,1/1/9999,1/1/9999,8/22/2020,...,35990003,DEL C/S,9/19/1995,9/28/1994,50.9,Denied on 07/16/2020,21-Jul,NOT IN REVIEW PROCESS,2200,Drugs
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,4/21/2028,3/11/2043,10/20/2020,...,35990003,POSS CONT SUB PG1 4-200 WITD,3/12/2018,3/31/2017,25,Denied on 08/13/2020,21-Aug,NOT IN REVIEW PROCESS,2200,Drugs
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,8/16/2025,6/27/2040,2/14/2018,...,35990003,DEL CONT SUB 4G-200G,11/10/2015,5/5/2015,25,Denied on 02/06/2020,21-Feb,NOT IN REVIEW PROCESS,2200,Drugs


In [20]:
rcbinlookup_df["Sentence_Years"].unique()

array(['8', '25', '50.9', '12', '17', '15', '10', '18', '65', '22', '30',
       '35', '40', '60', '5', '9', '50', '16', '20', '3', '7', '23', '75',
       '70', '99', '14', '6', '45', '54', '13', '26', '28', '19', '24',
       '85', '33', '27', '49', '11', '29', '32', '55', '80', '9.5', '21',
       '13.5', '68', '90', '48', '78', '8.5', '37', '95', '4', '42', '43',
       '56', '38', '36', '12.5', '47', '2', '15.5', '77', '44', '0.7',
       '10.5', '100.9', '61', '2.5', '3.3', '12.6', '6.5', '3.5', '10.1',
       '1.5', '52', '7.5', '41', '52.5', '57', '31', '72', '53', '22.5',
       '17.5', '32.5', '62', '17.6', '130', '39', '101.9', '198', '51',
       '119', '3000', '1000', '159', '98', '102', '114', '50.5', '58',
       '106', '46', '297', '67', '66', '104', '88', '69', '161', '76',
       '87', '146', '91', '37.5', '17.4', '82', '73', '63', '89', '27.5',
       '92', '34', '39.5', '18.5', '81', '26.5', '47.5', '59', '74',
       '16.5', '25.5', '18.3', '6.2', '14.5', '2024.9',

In [21]:
#these are global changes - so need to check at some point for data in other columns being inadvertently changed
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.9",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2092.7", value="8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2027.0", value = "10.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2026.4", value = "11.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2027.1",  value = "10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.8",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.7", value="7.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.5", value = "5.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2026.3", value = "8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.8",  value = "5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.4",value="8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2019.8", value="10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.2", value = "5.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.0", value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2029.9",  value = "10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.3",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2027.9", value="8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.4", value = "6.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.4", value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.5",value="2.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.9", value="3.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.8", value = "4.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.0", value = "5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.3",  value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.6",value="4.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.2", value="5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.7", value = "5.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.9", value = "4.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2025.1",  value = "9.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.5",value="1.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2019.5", value="4.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2029.5", value = "10.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2021.9", value = "10.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2025.0",  value = "6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.5",value="5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2022.6", value="3.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.2", value = "2.0") 
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2030.0", value = "11.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2023.6",  value = "5.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2024.2",value="6.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2029.7",  value = "8.0")
rcbinlookup_df=rcbinlookup_df.replace(to_replace="2020.9",value="2.0")

# rcbinlookup_df

In [22]:
rcbinlookup_df["Sentence_Years"].unique()

array(['8', '25', '50.9', '12', '17', '15', '10', '18', '65', '22', '30',
       '35', '40', '60', '5', '9', '50', '16', '20', '3', '7', '23', '75',
       '70', '99', '14', '6', '45', '54', '13', '26', '28', '19', '24',
       '85', '33', '27', '49', '11', '29', '32', '55', '80', '9.5', '21',
       '13.5', '68', '90', '48', '78', '8.5', '37', '95', '4', '42', '43',
       '56', '38', '36', '12.5', '47', '2', '15.5', '77', '44', '0.7',
       '10.5', '100.9', '61', '2.5', '3.3', '12.6', '6.5', '3.5', '10.1',
       '1.5', '52', '7.5', '41', '52.5', '57', '31', '72', '53', '22.5',
       '17.5', '32.5', '62', '17.6', '130', '39', '101.9', '198', '51',
       '119', '3000', '1000', '159', '98', '102', '114', '50.5', '58',
       '106', '46', '297', '67', '66', '104', '88', '69', '161', '76',
       '87', '146', '91', '37.5', '17.4', '82', '73', '63', '89', '27.5',
       '92', '34', '39.5', '18.5', '81', '26.5', '47.5', '59', '74',
       '16.5', '25.5', '18.3', '6.2', '14.5', '6.0', '8

In [23]:
rcbinlookup_df.loc[rcbinlookup_df["Sentence_Years"] == "800.0"]

,SID_Number,TDCJ_Number,Name,Current_Facility,Gender,Race,Age,Projected_Release,Max_Sentence_Date,Parole_Eligible_Date,...,Offense_Code,TDCJ_Offence,Sentence_Date,Offense_Date,Sentence_Years,Last_Parole_Decision,Next_Parole_Review,Parole_Review_Status,Offense_Category,Bin_Cat


In [24]:
# nanloc_df = rcbinlookup_df[pd.to_numeric(rcbinlookup_df['Sentence_Years'], errors='coerce').isnull()]
# nanloc_df

# 	SID_Number	TDCJ_Number	Name	Current_Facility	Gender	Race	Age	Projected_Release	Max_Sentence_Date	Parole_Eligible_Date	...	Offense_Code	TDCJ_Offence	Sentence_Date	Offense_Date	Sentence_Years	Last_Parole_Decision	Next_Parole_Review	Parole_Review_Status	Offense_Category	Bin_Cat
# 71997	4340086	2319589	DAVIS,CHARLES ELLIS	Estelle	M	W	49	06/29/2021	NaN	NaN	...	35990015	POSS CONT SUB 1-4G	04/14/2020	07/30/2019	NaN	NaN	NaN	NaN	2200	Drugs



In [25]:
rcbinlookup_df.loc[71997,"Sentence_Years"] = 10.0

In [26]:
# rcbinlookup_df["Sentence_Years"].unique()

In [27]:
# Create age bins
# must have 1 extra bin then actual catorgies - hence the 0 at the beginning

agebins = [0, 19.9, 29.9, 39.9, 49.9, 59.9, 69.9, 79.9, 89.9, 99.9, 199.9]

# Create labels for these bins
agegroup_labels = ["Less_Than_20", "20_to_30", "30_to_40",
                   "40_to_50", "50_to_60", "60_to_70",
                   "70_to_80", "80_to_90", "90_to_100", "Greater_than_100"]   

In [28]:
# Create new column - Age_Bin

rcbinlookup_df["Age_Bin"] = pd.cut(rcbinlookup_df["Age"], agebins, labels=agegroup_labels)
rcbinlookup_df.head()

,SID_Number,TDCJ_Number,Name,Current_Facility,Gender,Race,Age,Projected_Release,Max_Sentence_Date,Parole_Eligible_Date,...,TDCJ_Offence,Sentence_Date,Offense_Date,Sentence_Years,Last_Parole_Decision,Next_Parole_Review,Parole_Review_Status,Offense_Category,Bin_Cat,Age_Bin
0,234378,2174505,"LITTLE,AVA JESSUP",Young,F,W,75,10/1/2020,11/15/2025,9/26/2018,...,POSS CS 4-200G WID METH,12/14/2017,9/13/2017,8,Approved on 07/01/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
1,1116232,1366652,"SIDLE,JOHN ARTHUR",Pack,M,W,78,5/2/2021,6/18/2033,3/8/2016,...,UNLAW POSS WIT C/S-METHAM,4/6/2006,9/3/2004,25,Approved on 09/17/2020,NaN,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
2,1409034,729430,"LERMA,GILBERT LOPEZ",Wynne,M,H,70,1/1/9999,1/1/9999,8/22/2020,...,DEL C/S,9/19/1995,9/28/1994,50.9,Denied on 07/16/2020,21-Jul,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
3,1468825,2205051,"EVANS,DAYTON BUD",Pack,M,W,70,4/21/2028,3/11/2043,10/20/2020,...,POSS CONT SUB PG1 4-200 WITD,3/12/2018,3/31/2017,25,Denied on 08/13/2020,21-Aug,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80
4,1473183,2038325,"GREEN,ROGER DALE SR",Duncan,M,W,73,8/16/2025,6/27/2040,2/14/2018,...,DEL CONT SUB 4G-200G,11/10/2015,5/5/2015,25,Denied on 02/06/2020,21-Feb,NOT IN REVIEW PROCESS,2200,Drugs,70_to_80


In [29]:
## Cleanup Data Column for Counties 

rc_un = rcbinlookup_df['County'].unique()
rc_un = np.sort(rc_un)
rc_un = pd.DataFrame(rc_un)
rc_un.to_csv('../Resources/rc_counties.csv', index=False)


In [30]:
# Replace Lasalle with La Salle

rcbinlookup_df = rcbinlookup_df.replace(to_replace="Lasalle", value= "La Salle")

In [31]:
# Loading and manipulating population data for 2020 pop data 

countyall_df = pd.read_csv('../Resources/pop_data_with_projection.csv')
texas2020_df = countyall_df.query('year == 2020')
county2020_df = texas2020_df.loc[texas2020_df['age_group'] == "ALL"]
notex_df = county2020_df.drop([60])
proj_county2020 = notex_df.loc[:,["area_name", "total"]]
rcproj_county2020_df = proj_county2020.rename(columns={"area_name": "County", "total" : "Population_2020"})

pd.set_option('display.max_columns', None)
rcproj_county2020_df.head(65)


,County,Population_2020
306,Anderson,58199
552,Andrews,22269
798,Angelina,90437
1044,Aransas,27699
1290,Archer,8344
...,...,...
15066,Denton,897953
15312,Dewitt,21737
15558,Dickens,2174
15804,Dimmit,11743


In [33]:
# Merge Prison and Population database
mother_ship = pd.merge_ordered(rcbinlookup_df, rcproj_county2020_df, fill_method='ffill', left_by='County')
mother_ship.isna().sum()
values = {"Population_2020": 21737}
mother_ship.fillna(value=values)
mother_ship.isna().sum()
#county_error = mother_ship[mother_ship['Population_2020'].isnull()]
#county_error.head()
#county_error = county_error['County'].unique()

SID_Number                  0
TDCJ_Number                 0
Name                        0
Current_Facility            0
Gender                      0
Race                        0
Age                         0
Projected_Release           0
Max_Sentence_Date          28
Parole_Eligible_Date     9017
Case_Number                99
County                      0
Offense_Code                0
TDCJ_Offence                0
Sentence_Date            1233
Offense_Date                0
Sentence_Years              0
Last_Parole_Decision     7815
Next_Parole_Review      22397
Parole_Review_Status     7815
Offense_Category            0
Bin_Cat                     0
Age_Bin                     0
Population_2020             0
dtype: int64

In [36]:
%store mother_ship

Stored 'mother_ship' (DataFrame)
